# Trabalho 1 - EEL891

Etapas que serão realizadas a seguir:
- Implementar o pré-processamento
- Visualizar, analisar e selecionar os atributos
- Escolher os métodos que serão usados e compará-los
- Verificar o desempenho com validação cruzada
- Ajustas os hiperparâmetros para chegar ao melhor resultado possível
- Treinar o modelo com o conjunto de teste

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from scipy.stats import pearsonr

### Pré-processamento

Agora, o próximo passo é importar o conjunto de treinamento:

In [ ]:
dados = pd.read_csv(r"C:\Users\andre\Documents\UFRJ\SÉTIMO PERÍODO\Introdução ao Aprendizado de Máquina\EEL891-Trabalho-1\conjunto_de_treinamento.csv")
dados = dados.iloc[:,:]

Embaralhamento dos dados e separá-los em atributos e alvo (convertendo para um array):

In [ ]:
dados = dados.sample(frac=1)
x = dados.iloc[:,:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

Descartando algumas informações que não são relevantes. O campo "id_solicitante" é um número de 1 a 20000, ou seja, apenas atrapalharia o modelo. Os campos "grau_instrucao" e "possui_telefone_celular" não foram extraídos corretamente, então segui a orientação do professor para descartá-los. E o campo "qtde_contas_bancarias_especiais" é exatamente igual ao campo "qtde_contas_bancarias", logo foi descartado seguindo a orientação do professor.

In [ ]:
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'possui_telefone_celular', 'qtde_contas_bancarias_especiais'],axis=1)

Identificação das variáveis categóricas olhando o dicionário de dados:

In [ ]:
variaveisCategoricas = [x for x in dados.columns if dados[x].dtype=='object' or x == 'produto_solicitado' or x == 'tipo_endereco' or x =='estado_civil' or  x =='nacionalidade' or x=='tipo_residencia' or x=='possui_email' or x=='possui_cartao_visa' or x=='possui_cartao_mastercard' or x=='possui_cartao_diners' or x=='possui_cartao_amex' or x=='possui_cartao_cartoes' or x=='possui_carro' or x=='profissao' or x=='ocupacao' or x=='profissao_companheiro' or x=='local_onde_reside' or x=='local_onde_trabalha']
print(variaveisCategoricas)

Verificando as variáveis categóricas (quantidade de categorias e quais são):

In [ ]:
for v in variaveisCategoricas:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")

Correção de alguns valores (principalmente em branco). A ideia inicial é criar uma nova categoria para os valores em branco, e depois tentar excluí-los para ver qual é a resposta dos modelos:

In [ ]:
for x in variaveisCategoricas:
    if x == 'sexo':
        dados[x].replace(' ', 'N', inplace=True)
    if x == 'estado_onde_nasceu':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_residencial':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'tipo_residencia':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'estado_onde_trabalha':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_trabalho':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'profissao':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'ocupacao':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'profissao_companheiro':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")

In [ ]:
for x in dados:
    if x == 'meses_na_residencia':
        dados[x].fillna(300, inplace=True)
    if x == 'grau_instrucao_companheiro':
        dados[x].fillna(6, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")

Binarizar e fazer o one hot encoding de alguns parâmetros:

In [ ]:
binarizador = LabelBinarizer()
for x in ['tipo_endereco', 'possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']:
    dados[x] = binarizador.fit_transform(dados[x])
#print(dados.head(4).T)

a=0
for x in dados:
    for y in x:
    #print(dados[x].isna())
        print(pd.isnull(y))
    if dados[x].isna().values[0] == True:
        a+=1
        print(x, ' = ', a)
# dados = pd.get_dummies(dados, columns=[])

Verificar a distribuição entre os possíveis valores dentro dos parâmetros:

In [ ]:
for x in dados:
    print(dados[x].value_counts())

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [ ]:
colunas = dados.columns
for col in colunas:
    if type(dados[col][0]) != str:
        corr = pearsonr(dados[col], dados['inadimplente'])
        print(col, " = %.4f" % corr[0], " , %.10f" % corr[1])

In [ ]:
#atributos usados pelo classificador
atributosSelecionados = [
    'produto_solicitado',
    'dia_vencimento',
    'forma_envio_solicitacao',
    'tipo_endereco',
    'sexo',
    'idade',
    'estado_civil',
    'qtde_dependentes',
    'nacionalidade',
    'estado_onde_nasceu',
    'estado_onde_reside',
    'possui_telefone_residencial',
    'codigo_area_telefone_residencial',
    'tipo_residencia',
    'meses_na_residencia',
    'possui_telefone_celular',
    'possui email',
    'renda_mensal_regular',
    'renda_extra',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_cartao_cartoes',
    'qtde_contas_bancarias',
    'valor_patrimonio_pessoal',
    'possui_carro',
    'vinculo_formal_com_empresa',
    'estado_onde_trabalha',
    'possui_telefone_trabalho',
    'codigo_area_telefone_trabalho',
    'meses_no_trabalho',
    'profissao',
    'ocupacao',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    'local_onde_reside',
    'local_onde_trabalha'
]

dados = dados[atributosSelecionados]

#embaralhar o conjunto de dados
dadosEmbaralhados = dados.sample(frac=1, random_state=1)

#criar arrays x e y separando os parâmetros do alvo
x = dadosEmbaralhados.loc[:,dadosEmbaralhados.columns!='inadimplente'].values()
y = dadosEmbaralhados.loc[:,dadosEmbaralhados.columns=='inadimplente'].values()

#separar em conjunto de treino e de teste
amostrasTreinamento = 15000

xTreino = x[:amostrasTreinamento,:]
yTreino = x[:amostrasTreinamento].ravel()

xTeste = x[amostrasTreinamentotrasTreinamento:,:]
yTeste = y[amostrasTreinamento:].ravel()

#ajustar a escala
scaler = MinMaxScaler()
scaler.fit(xTreino)

xTreino = scaler.transform(xTreino)
xTeste = scaler.transform(xTeste)

#treinar classificador KNN
classificador = KNeighborsClassifier(n_neighbors=5)

classificador = classificador.fit(xTreino, yTreino)

#obter as respostas do classificador
yRespostaTreino = classificador.predict(xTreino)
yRespostaTeste = classificador.predict(xTeste)

#desempenho na amostra de treino
total = len(yTreino)
acertos = sum(yRespostaTreino==yTreino)
erros = sum(yRespostaTreino!=yTreino)

print("Total de amostras: ", total)
print("Respostas corretas: ", acertos)
print("Respostas erradas: ", erros)

acuracia = acertos / total

print("Acurácia: %.1f %%" % (100*acuracia) )

#verificar a variacao da acuracia com o numero de vizinhos
print("\n  K  TREINO  TESTE")
print(" --  ------  ------")

for k in range(1,50):
    classificador = KNeighborsClassifier(
        n_neighbors=k,
        weights = 'uniform',
        p=1
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%3d" % k,
        "%6.1f" % (100*acuraciaTreino)
        "%6.1f" % (100*acuraciaTeste)
    )
    

#verificar a variacao da acuracia para regressao logistica
print("\n         C  TREINO  TESTE")
print(" ----------  ------  ------")

for k in range(-6,7):
    c = pow(10,k)
    classificador = LogisticRegression(
        penalty = 'l2',
        C = c
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%14.6f" % c,
        "%6.1f" % (100*acuraciaTreino)
        "%6.1f" % (100*acuraciaTeste)
    )

### Treinamento do modelo

Classificador Bayesiano com Distribuição Multinomial:

In [ ]:
classificador = MultinomialNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Classificador Bayesiano com Distribuição Multinomial:

In [ ]:
classificador = GaussianNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))